In [7]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests
import shutil
import yfinance
from bs4 import BeautifulSoup

In [197]:

class SECSubmissions:
    def __init__(self,ticker):
        self.ticker = ticker 
        self.CIK = self.getCIK()
        self.filings = self.getSECFilings(self.CIK)
        self.filings = self.findDate(self.filings)
        self.fiscalYear = self.findFiscalYear(self.filings)
        self.filings = self.mergeFiscalYearWithFilings(self.filings, self.fiscalYear)
        self.filings = self.createSubmissionName(self.filings)
        self.printUrl(self.filings)
       
        
    def getCIK(self): #get SEC code for company based on ticker
        headers={"User-Agent": "Mozilla/5.0"}
        symbol_to_cik = requests.get("https://www.sec.gov/files/company_tickers.json").json() #returns a json dictionary with a indexed list of all different companies 
        ciks = {info["ticker"]:info["cik_str"] for key,info in symbol_to_cik.items()} #create dictionary indexable by ticker
        return ciks[self.ticker]
    
        
    #input: CIK
    #request the data on every submission to SEC website for the company
    def getSECFilings(self,CIK):
        headers={"User-Agent": "Mozilla/5.0"}
        edgar_filings = requests.get(f"https://data.sec.gov/submissions/CIK{CIK:0>10}.json", headers=headers).json()
        filings = pd.DataFrame(edgar_filings["filings"]["recent"])
        filings = filings.loc[filings["reportDate"]>"2014-01-01"]#filings pre-2014 are unable to be downloaded due to different excel format
        filings = filings.loc[filings["form"].isin(["10-Q","10-K"])].reset_index(drop=True) #drop all filings that are not 10Q or 10K
        return filings 
    
    #input filings
    #output filings with mo and year
    def findDate(self,filings):
        filings["Month"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%m").astype("int") #get Month of all filings 
        filings["Year"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%Y").astype("int") #get Year of all filings
        filings["Date"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%m/%d/%Y")
        return filings
    
    #input a list of 10k filings with mo and year
    #find one fiscal year of data via 10k, then find all possible months filed and organize to fiscal period
    def findFiscalYear(self,filings):
        #get a fiscal Year of data by finding the last 10K and the three 10Qs before it
        TenKIndex = filings.loc[filings["form"]=="10-K"].index[0] 
        fiscalYearKey = filings.loc[TenKIndex:TenKIndex+3,["Month","form","Date"]].copy().reset_index(drop=True)
        fiscalYearKey["Period"] = ["Year Ended", "Q3","Q2","Q1"]
        
        #fiscal year data for certain period can vary between months ie. both Jan & Feb, therefore I found all possible months of filings and found appropriate period for each month
        fiscalYear = pd.DataFrame({"Month":filings.Month.unique()})
        fiscalYear.index = list(fiscalYear.Month.apply(lambda x: np.argmin(np.abs(fiscalYearKey.Month-x)))) #using fiscal year key, I found the index of the period in fiscalYearKey with closest month to each month 
        fiscalYear = pd.merge(fiscalYear,fiscalYearKey, left_index=True, right_index=True, suffixes=("","_x")) #I merged together together all of the possible months with their corresponding period based on the key
        fiscalYear = fiscalYear.drop(columns="Month_x")
        
        return fiscalYear
    
    #input filings of 10k/q
    #merge by month to find which fiscal period, if filing of 10q is later in year than 10k, then adjust year so it is a part of right fiscal year
    def mergeFiscalYearWithFilings(self, filings, fiscalYear):
        #merge fiscal year labels with filings by the month that they were reported
        filings = pd.merge(filings,fiscalYear,on=["Month","form"],how="left")    
        #set Fiscal Year
        filings["Fiscal Year"] = filings["Year"]
        #get Quarters that are a part of different fiscal Year than report date
        TenKMo = fiscalYear.loc[fiscalYear["form"]=="10-K"]["Month"].iloc[0]
        filings.loc[(filings["Month"]>TenKMo)&(filings["form"]!="10-K"),"Fiscal Year"] += 1 #the 10K is reported in a month before the 10Q so therefore the 10Q is the next fiscal year 
        
        return filings
    
    

    #input filings
    #take the fiscal period and year and create a string, if different fiscal Year than filing date use parenthesis
    def createSubmissionName(self,filings):
        filings["Name"] = filings.apply(self.submissionName,axis=1)
        return filings
        
    def submissionName(self,row):
        period = row["Period"]
        fiscalYear = row["Fiscal Year"]
        Year = row["Year"]
        if fiscalYear != Year:
                name = f"{period} {fiscalYear} ({Year})"
        else: 
                name = f"{period} {fiscalYear}"
        return name
    
    
    #input filings
    #request the filing from SEC, print link and save to computer
    def printUrl(self,filings):
        self.path =  os.path.join("../input",f"Financial Statement {self.ticker}")
        self.folder = os.path.join(self.path,"docs")
        
        for i,row in filings.iterrows():
            name = row["Name"]
            accessionNum = row["accessionNumber"].replace("-","")
            doc = row["primaryDocument"]
            url = f"https://www.sec.gov/Archives/edgar/data/{self.CIK}/{accessionNum}/{doc}"
            print(f"{name} {url}")
            req = requests.get(url,headers={"User-Agent": "Mozilla/5.0"})
            edgar_str = req.text
            file = open(os.path.join(self.folder,f"{name}.htm"),"w")
            file.write(edgar_str)
           
        

In [199]:
sub = SECSubmissions("AAPL")
sub.printUrl()

Q3 2022 https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/aapl-20220625.htm
Q2 2022 https://www.sec.gov/Archives/edgar/data/320193/000032019322000059/aapl-20220326.htm
Q1 2022 (2021) https://www.sec.gov/Archives/edgar/data/320193/000032019322000007/aapl-20211225.htm
Year Ended 2021 https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/aapl-20210925.htm
Q3 2021 https://www.sec.gov/Archives/edgar/data/320193/000032019321000065/aapl-20210626.htm
Q2 2021 https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/aapl-20210327.htm
Q1 2021 (2020) https://www.sec.gov/Archives/edgar/data/320193/000032019321000010/aapl-20201226.htm
Year Ended 2020 https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926.htm
Q3 2020 https://www.sec.gov/Archives/edgar/data/320193/000032019320000062/aapl-20200627.htm
Q2 2020 https://www.sec.gov/Archives/edgar/data/320193/000032019320000052/a10-qq220203282020.htm
Q1 2020 (2019) https://www.sec.gov/Archives/e

In [4]:
class HelperFunctions():
    #input: a list of quarters such as "Q2 2022"
    #iterates through a finds a date/year in the file
    def getYears(self,quarters): 
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    
    def getUniqueYears(self,quarters):
        return set(self.getYears(quarters))
    
    #input: compiledStatement
    #returns all quarters but removes first col of label
    def getQuarters(self, compiledStatement): 
        return list(compiledStatement.columns[1:])
    
    def getPossibleFilingsFromYears(self, years): 
        possible = [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"]]
        possible += [q + " " + str(year) for year in years for q in ["6mo","9mo"]]
        return possible
    
    #input: compiledStatement
    #find all years in statement and get all possible filings in order then only keep the ones in actually in the statement
    def reorderQuarters(self,compiledStatement): 
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        cols = self.getPossibleFilingsFromYears(years)
        cols = [col for col in cols if col in quarters]
        compiledStatement = compiledStatement[["label"] + cols]
        return compiledStatement
    
    
    
    

In [212]:

class FindStockPrice:
    def __init__(self,ticker, fromDate=2017,endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path =  os.path.join("../input",f"Financial Statement {self.ticker}")
        self.file = os.path.join(self.path,"Stock Price.xlsx")
        self.fiscalYearFile = os.path.join(self.path,"fiscalYear.csv")
        
        self.fiscalYear, self.day = self.readFiscalYear(self.fiscalYearFile)
        self.FinalStockPrices = self.collectStockPriceData()
        
        self.FinalStockPrices.to_excel(self.file,index=False)
    
    #input: fiscal year file 
    #output: a series with each period and the month corresponding, as well as the day of month of filing
    def readFiscalYear(self,fiscalYearFile):
        fiscalYear = pd.read_csv(fiscalYearFile)
        #select the day for fiscal year filings (by grabbing day of first period in fiscal year)
        day = pd.to_datetime(fiscalYear["Date"]).dt.strftime("%d")[0]
        #find month of fiscal period based on the date col because month can vary whereas date is consistent throughout period
        fiscalYear.Month = pd.to_datetime(fiscalYear["Date"]).dt.strftime("%m").astype("int")
        fiscalYear = fiscalYear.groupby("Period")["Month"].first()
        return fiscalYear, day
        
    #go through each year and find stock price for each quater
    def collectStockPriceData(self):
        FinalStockPrices = pd.DataFrame({"label":["Stock Price"]})
        TenKMo = self.fiscalYear.loc["Year Ended"]
        
        date = f"{self.fromDate-1}-{TenKMo:02d}-{self.day}"
        price = yfinance.download(self.ticker,start=date,progress=False)["Close"].iloc[0]
        FinalStockPrices[f"Year Ended {self.fromDate-1}"] = [price]
        
        for fiscalYear in range(self.fromDate, self.endDate+1):
            for period,mo in self.fiscalYear.iteritems():
                year = fiscalYear
                #if quater comes after year ended, then the year of filing is a year before the technical fiscal year
                if mo > TenKMo:
                    year = year-1
                date = f"{fiscalYear}-{mo:02d}-{self.day}"
                
                #in doing this current fiscal year, if date is in future period, stop
                if datetime.strptime(date, "%Y-%m-%d") > datetime.now():
                    break
                    
                price = yfinance.download(self.ticker,start=date,progress=False)["Close"].iloc[0]
                if period=="Year Ended":
                    FinalStockPrices[f"Q4 {year}"] = [price]
                FinalStockPrices[f"{period} {year}"] = [price]
            
        return FinalStockPrices
        


In [213]:
FindStockPrice("AAPL")

In [304]:
yfinance.download("AAPL",start="2016-09-25",progress=False)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-09-26,27.910000,28.347500,27.887501,28.219999,26.354940,119477600
2016-09-27,28.250000,28.295000,28.084999,28.272499,26.403969,98429600
2016-09-28,28.422501,28.660000,28.357500,28.487499,26.604757,118564400
2016-09-29,28.290001,28.450001,27.950001,28.045000,26.191505,143548000
2016-09-30,28.115000,28.342501,27.950001,28.262501,26.394630,145516400
...,...,...,...,...,...,...
2022-07-29,161.240005,163.630005,159.500000,162.509995,162.509995,101689200
2022-08-01,161.009995,163.589996,160.889999,161.509995,161.509995,67829400
2022-08-02,160.100006,162.410004,159.630005,160.009995,160.009995,59907000


In [317]:
class WriteSales(SECSubmissions, HelperFunctions):
    def __init__(self,ticker,Type):
        super().__init__(ticker)
        self.path =  os.path.join("../input",f"Financial Statement {self.ticker}")
        self.file = os.path.join(self.path,"Sales By Segment.xlsx")
        
        #grab dictionary with all the tables by segment, so as not to replace entire excel document
        self.Excel = self.readExcel()
        
        self.type = Type
        self.Excel[self.type] = self.writeSalesData(self.filings)
        self.writeExcel()
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.file)
        for name, table in self.Excel.items():  
            table.to_excel(Excel, index=False, sheet_name=name)
        Excel.save()
        
    def readExcel(self):
        if not os.path.isfile(self.file): return {}
        Excel = pd.read_excel(self.file,sheet_name=None)
        return Excel
        
    #input: the html string from the raw 10q/k, key term strings 
    #filter through tables and only keep last table that has all keyterm strings
    #grab the first number in each row for the value (ie sales), grab the first item in each row for the label (ie. product)
    #output: parsed table with label and value
    def filterTables(self,edgar_str,strings):
        finaltable = False
        soup = BeautifulSoup(edgar_str, 'html.parser')
        for table in soup.find_all('table'):
            table = [list(tr.stripped_strings) for tr in table.find_all('tr') if tr.text]
            table_string = ""
            for row in table: 
                for col in row: table_string+=col
            if all([s.lower() in table_string.lower() for s in strings]):
                finaltable = table
        if not finaltable: return False
#         for row in finaltable: print(row)
        if self.type=="Debt":
            indexes = [ [[i,row] for i,value in enumerate(row) if len(re.findall("\d{4}",value))>0 ] for row in finaltable]
            indexes = [indexes[i][-1] if len(indexes[i])>0 else [0,row] for i,row in enumerate(finaltable)]
            finaltable = [[" ".join(row[:i+1])]+row[i+1:] for i, row in indexes]
#         for row in finaltable: print(row)
        labels = [row[0] for row in finaltable]
        finaltable = [row for row in finaltable if row!=[]]
        finaltable = [[x.strip("(") for x in row if x!="%" and x!="$" and "("!=x and ")" not in x] for row in finaltable]
        
        
        nums = [[x for x in row if len(re.findall("[0-9]",x))>0 and len(re.findall("[a-zA-Z\-]",x))==0] for row in finaltable]
        nums = [row[0] if len(row)>0 else np.nan for row in nums]
        
        finaltable = list(zip(labels,nums))
        
       
        return finaltable
    #input: embedded list of table with the label and the values. for ex. sales by product
    #convert into dataframe, with name of the quarter, make numeric values an integer by removing commas 
    def tableToFrame(self,table,name):
        df = pd.DataFrame(table).iloc[:,[0,1]]
        df.columns = ["label",name]
        df[name] = pd.to_numeric(df[name].replace(r",","",regex=True))
        return df
    
    #input: the html string from the raw 10q/k
    #iterate through tables for data on products, trim table by starting and ending index, convert embedded lists into dataframe
    #table with sales by product
    def parseForProductsTable(self,edgar_str, name):
        table = self.filterTables(edgar_str,["iPhone","Total net sales"])
        
        indexstart = [i for i, row in enumerate(table) if "iPhone" in row[0]][0]
        table = table[indexstart:]
        indexend = [i for i, row in enumerate(table) if "Total net sales" in row[0]][0]
        table = table[:indexend+1]

        table = self.tableToFrame(table,name)
        return table 
    
    def parseForUnitsTable(self,edgar_str, name):
        table = self.filterTables(edgar_str,["Unit Sales by Product:"])
        if not table: return pd.DataFrame()
        table = [row for row in table if row!=[]]
        indexstart = [i for i, row in enumerate(table) if "Unit Sales by Product:" in row[0]][0]
        table = table[indexstart:]
        table = self.tableToFrame(table,name)
        return table 
    
    def parseForGeographyTable(self,edgar_str, name):
        table = self.filterTables(edgar_str,["Americas","Total net sales"])
        
        indexstart = [i for i, row in enumerate(table) if "Americas" in row[0]][0]
        table = table[indexstart:]
        indexend = [i for i, row in enumerate(table) if "Total net sales" in row[0]][0]
        table = table[:indexend+1]
        
        table = self.tableToFrame(table,name)
        return table 
    
    
    def parseForCapitalTable(self,edgar_str, name):
        table = self.filterTables(edgar_str,["Gross property, plant and equipment"])
        
        if not table: return pd.DataFrame()
        indexstart = 1
        table = table[indexstart:]
        table = self.tableToFrame(table,name)
        return table 
    
    
    def parseForLeaseTable(self,edgar_str, name):
        table = self.filterTables(edgar_str,["Operating leases","Total lease liabilities"])

        if not table: return pd.DataFrame()
        indexstart = 1
        table = table[indexstart:]
        
        table = self.tableToFrame(table,name)
        table["label"] = [l + str(i) for i, l in enumerate(list(table.label))]
        return table 

    def parseForDebtTable(self,edgar_str, name):
        table = self.filterTables(edgar_str,["debt issuances","Total term debt"])

        if not table: return pd.DataFrame()
        indexstart = 1
        table = table[indexstart:]
        print(table)
        table = self.tableToFrame(table,name)
        table = table.rename(columns={"label":"label "+name})

        
        return table 
    
    #input: filings
    #iterate through and use the accession number to grab filings from SEC
    #for each type, filter through tables in filing to find correct table, compile the data from each quarter into a dataframe 
    #output: dataframe with each quarter and the sales or other values by segment 
    def writeSalesData(self,filings):
        finalSales = pd.DataFrame(columns=["label"])

        for i,row in filings.iterrows():
            name = row["Name"]
            if row["Fiscal Year"]<=2016 and name != "Year Ended 2016":
                break
            name = re.sub("\s\([\w\W]+?\)","",name)
            accessionNum = row["accessionNumber"].replace("-","")
            doc = row["primaryDocument"]
            url = f"https://www.sec.gov/Archives/edgar/data/{self.CIK}/{accessionNum}/{doc}"
            print(url)
            req = requests.get(url,headers={"User-Agent": "Mozilla/5.0"})
            edgar_str = req.text
            
            if self.type == "Geography":
                sales = self.parseForGeographyTable(edgar_str, name)
            
            if self.type == "Product":
                sales = self.parseForProductsTable(edgar_str, name)
            
            if self.type == "Units":
                sales = self.parseForUnitsTable(edgar_str, name)
                
            if self.type == "Capital":
                sales = self.parseForCapitalTable(edgar_str, name)
                
            if self.type == "Lease":
                sales = self.parseForLeaseTable(edgar_str, name)
                
            if self.type == "Debt":
                sales = self.parseForDebtTable(edgar_str, name)
                finalSales = pd.concat([finalSales, sales], axis=1)
                
                continue
                

            if self.type == "Total Term Debt":
                sales = self.parseForDebtTable(edgar_str, name)
                if not sales.empty:
                    sales = sales.rename(columns={"label "+name:"label"})
                    sales = sales.loc[sales["label"].str.contains("Total term debt")]
                
            if not sales.empty:
                finalSales = pd.merge(finalSales, sales, on=["label"], how="outer")
            
        
        if self.type in ["Capital","Debt","Total Term Debt"]:
            finalSales =self.performBalanceMath(finalSales)
        else:
            finalSales = self.performIncomeMath(finalSales)
        
        if self.type not in ["Debt"]:
            finalSales = self.reorderQuarters(finalSales)
        

        return finalSales
    
    def performBalanceMath(self, compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = compiledStatement["Year Ended "+year]
        return compiledStatement
    
    def performIncomeMath(self,compiledStatement): 
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
         #replace the nan values with 0 so that the subtraction between an nan value does not result in nan
        c = compiledStatement.copy().replace(r'\s+', np.nan, regex=True).fillna(0).drop(columns="label")
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = c["Year Ended "+year] - c["Q3 "+year] - c["Q2 "+year] - c["Q1 "+year]
        
        compiledStatement = compiledStatement.replace(0,np.nan)
        return compiledStatement
            
        

In [318]:
obj = WriteSales("AAPL","Debt")

https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/aapl-20220625.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019322000059/aapl-20220326.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019322000007/aapl-20211225.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/aapl-20210925.htm
[('Amount', nan), ('2013 – 2020 debt issuances:', nan), ('Floating-rate notes 2022', '1,750'), ('Fixed-rate 0.000 % – 4.650 % notes 2022 – 2060', '95,813'), ('Second quarter 2021 debt issuance:', nan), ('Fixed-rate 0.700 % – 2.800 % notes 2026 – 2061', '14,000'), ('Fourth quarter 2021 debt issuance:', nan), ('Fixed-rate 1.400 % – 2.850 % notes 2028 – 2061', '6,500'), ('Total term debt', '118,063'), ('Unamortized premium/(discount) and issuance costs, net', '380'), ('Hedge accounting fair value adjustments', '1,036'), ('Less: Current portion of term debt', '9,613'), ('Total non-current portion of term debt', '109,106')]
https://www.sec.gov/Archives/edgar/da

[('Amount', nan), ('2013 debt issuance of $17.0 billion:', nan), ('Fixed-rate 2.400%\xa0–\xa03.850% notes 2023 – 2043', '8,500'), ('', nan), ('2014 debt issuance of $12.0 billion:', nan), ('Floating-rate notes 2019', '1,000'), ('Fixed-rate 2.100%\xa0–\xa04.450% notes 2019 – 2044', '8,500'), ('', nan), ('2015 debt issuances of $27.3 billion:', nan), ('Floating-rate notes 2019 – 2020', '1,497'), ('Fixed-rate 0.350%\xa0–\xa04.375% notes 2019 – 2045', '24,181'), ('', nan), ('2016 debt issuances of $24.9 billion:', nan), ('Floating-rate notes 2019 – 2021', '1,350'), ('Fixed-rate 1.100%\xa0–\xa04.650% notes 2019 – 2046', '23,038'), ('', nan), ('2017 debt issuances of $28.7 billion:', nan), ('Floating-rate notes 2019 – 2022', '3,250'), ('Fixed-rate 0.875%\xa0–\xa04.300% notes 2019 – 2047', '25,454'), ('', nan), ('2018 debt issuance of $7.0 billion:', nan), ('Fixed-rate 1.800%\xa0–\xa03.750% notes 2019 – 2047', '7,000'), ('Total term debt', '103,770'), ('', nan), ('Unamortized premium/(discoun

[('Amount', nan), ('2013 debt issuance of $17.0 billion:', nan), ('Floating-rate notes 2018 2018', '2,000'), ('Fixed-rate 1.000%\xa0–\xa03.850% notes 2018 – 2043', '12,500'), ('', nan), ('2014 debt issuance of $12.0 billion:', nan), ('Floating-rate notes 2017 – 2019', '2,000'), ('Fixed-rate 1.050%\xa0–\xa04.450% notes 2017 – 2044', '10,000'), ('', nan), ('2015 debt issuances of $27.3 billion:', nan), ('Floating-rate notes 2017 – 2020', '1,785'), ('Fixed-rate 0.350%\xa0–\xa04.375% notes 2017 – 2045', '24,668'), ('', nan), ('2016 debt issuances of $24.9 billion:', nan), ('Floating-rate notes 2019 – 2021', '1,350'), ('Fixed-rate 1.100%\xa0–\xa04.650% notes 2018 – 2046', '23,616'), ('', nan), ('Second quarter 2017 debt issuance of $10.0 billion:', nan), ('Floating-rate notes 2019', '500'), ('Floating-rate notes 2020', '500'), ('Floating-rate notes 2022', '1,000'), ('Fixed-rate 1.550% notes 2019', '500'), ('Fixed-rate 1.900% notes 2020', '1,000'), ('Fixed-rate 2.500% notes 2022', '1,500'), 

In [ ]:
Operating leases